In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from scipy.stats import f_oneway
from scipy.stats import kruskal
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math
from matplotlib.colors import TwoSlopeNorm
import scipy
from matplotlib.cm import ScalarMappable
import matplotlib as mpl
from matplotlib.patches import Rectangle
mpl.rcParams['figure.dpi'] = 300

non_barcodes = ["Chondrocytes","EC-Arteriar","EC-Arteriolar","EC-Sinusoidal",
                        "Fibroblasts","MSPC-Adipo","MSPC-Osteo","Myofibroblasts","Osteo",
                        "Osteoblasts","Pericytes","Schwann-cells","Smooth-muscle", 
                       'CLP', 'lin-', 'MEP', 'CMP', 'GMP', 'MPP', 'HSC', 'B cell', 'Dendritic cells',
                        'Eo/Baso prog.', 'Ery prog.', 'Ery/Mk prog.', 'Erythroblasts', 'Gran/Mono prog.',
                        'LMPPs', 'Mk prog.', 'Mono prog.', 'Monocytes', 'NK cells', 'Neutro prog.', 'Neutrophils',
                        'T cells', 'large pre-B.', 'pro-B', 'small pre-B.']

type_to_ct = {'Non-hematopoietic':["Chondrocytes","EC-Arteriar","EC-Arteriolar","EC-Sinusoidal",
                        "Fibroblasts","MSPC-Adipo","MSPC-Osteo","Myofibroblasts","Osteo",
                        "Osteoblasts","Pericytes","Schwann-cells","Smooth-muscle"],
              'HPC' : ['CLP', 'lin-', 'MEP', 'CMP', 'GMP', 'MPP', 'HSC','Eo/Baso prog.', 'Ery prog.', 'Ery/Mk prog.',
                       'Gran/Mono prog.','LMPPs', 'Mk prog.', 'Mono prog.','Neutro prog.'],
              'Blood+Immune' : ['B cell', 'Dendritic cells','Erythroblasts','Monocytes', 'NK cells','Neutrophils','T cells', 'large pre-B.', 'pro-B', 'small pre-B.']}


hspc_dirs = ["general_niche/",
'general_niche_mpp/',
'general_niche_cmp/',
'general_niche_clp/',
'general_niche_gmp/',
'general_niche_mep/']

hspc_n = ["hsc",
'mpp',
'cmp',
'clp',
'gmp',
'mep']

ct_to_type = {}
for i in type_to_ct_HSPC.keys():
    for j in type_to_ct_HSPC[i]:
        ct_to_type[j] = i 
    
def process_csv(filename):
    pp = pd.read_csv(filename, header  = 0)
    pp = pp.set_index('Unnamed: 0')
    pp.index.names = [None]
    return pp

In [ ]:
bm_freq = pd.read_csv('bone_marrow_cell_frequencies.csv')
bm_freq = bm_freq.dropna(how='all')[['Cell Type', 'Frequency']]
bm_freq_dict = bm_freq.set_index('Cell Type').T.to_dict()

In [ ]:
bm_distance = pd.read_csv('bone_marrow_distancing.csv', index_col = 1)
bm_distance = bm_distance.dropna(how='all')
bm_distance = bm_distance.T.to_dict()

In [ ]:
file_names = [
                  "Chondrocytes", "ECArteriar", "ECArteriolar", "ECSinusoidal","Fibroblasts",
                  "MSPCAdipo", "MSPCOsteo", "Myofibroblasts", "Osteo", "Osteoblasts",
                  "Pericytes", "Schwanncells", "Smoothmuscle",
                  "CLP","lin","MEP","CMP","GMP","MPP","HSC",
                  "EoBaso prog","Ery prog","EryMk prog","GranMono prog",
                  "LMPPs","Mk prog","Mono prog","Neutro prog",
                  "B cell","Dendritic cells","Erythroblasts","Monocytes",
                  "NK cells","Neutrophils","T cells","large preB","proB","small preB"
                 ]

file_name_dict = {}
for i in range(len(file_names)):
    file_name_dict[file_names[i]] = non_barcodes_HSPC[i]

def get_specific_interaction(parent_path, n, cell_type, sorr):
    
    if sorr == 's':
        filename = 'hsc_to_' + cell_type
    else:
        filename = cell_type + "_to_hsc"
    
    interaction_df = process_csv(parent_path + 'all_pathways/' + filename + '.csv')
    
    return interaction_df

In [ ]:
def get_interaction_list(parent_dir, n, non_barcode_type = 'Non-hematopoietic', sorr = 'r'):


    non_barcodes = non_barcodes_HSPC
    type_to_ct = type_to_ct_HSPC


    # Cell Cell Contact
    filename = parent_dir + n + "_ccc_weights.csv"
    interaction_counts_ccc_r = process_csv(filename)
    
    interaction_counts_ccc_s = interaction_counts_ccc_r.T

    #Receiving
    interaction_counts_ccc_r = interaction_counts_ccc_r.loc[type_to_ct[non_barcode_type]]
    interaction_counts_ccc_r = interaction_counts_ccc_r.drop(columns=non_barcodes)

    #Sending
    interaction_counts_ccc_s = interaction_counts_ccc_s.loc[type_to_ct[non_barcode_type]]
    interaction_counts_ccc_s = interaction_counts_ccc_s.drop(columns=non_barcodes)


    # Secreted Signaling
    filename = parent_dir + n + "_ss_weights.csv"
    interaction_counts_ss_r = process_csv(filename)
    
    interaction_counts_ss_s = interaction_counts_ss_r.T

    #Receiving
    interaction_counts_ss_r = interaction_counts_ss_r.loc[type_to_ct[non_barcode_type]]
    interaction_counts_ss_r = interaction_counts_ss_r.drop(columns=non_barcodes)

    #Sending
    interaction_counts_ss_s = interaction_counts_ss_s.loc[type_to_ct[non_barcode_type]]
    interaction_counts_ss_s = interaction_counts_ss_s.drop(columns=non_barcodes)


    # ECM-R
    filename = parent_dir + n + "_ecmr_weights.csv"
    interaction_counts_ecmr_r = process_csv(filename)
    
    interaction_counts_ecmr_s = interaction_counts_ecmr_r.T

    #Receiving
    interaction_counts_ecmr_r = interaction_counts_ecmr_r.loc[type_to_ct[non_barcode_type]]
    interaction_counts_ecmr_r = interaction_counts_ecmr_r.drop(columns=non_barcodes)

    #Sending
    interaction_counts_ecmr_s = interaction_counts_ecmr_s.loc[type_to_ct[non_barcode_type]]
    interaction_counts_ecmr_s = interaction_counts_ecmr_s.drop(columns=non_barcodes)
    
    if sorr == 'r':
        int_list = [interaction_counts_ss_r, interaction_counts_ccc_r, interaction_counts_ecmr_r]
    else:
        int_list = [interaction_counts_ss_s, interaction_counts_ccc_s, interaction_counts_ecmr_s]
        
    return int_list

In [ ]:
def plot_population_hspcs_total(sorr, non_barcode_type, vmax = None, mask_lim = None,
                                frequency = None, imaging = None, saving=False):

    i = 0
    xticks = hspc_n
    xticks = [i.upper() for i in xticks]
    
    chosen_cmap = 'Reds'
    if sorr == 's':
        chosen_cmap = 'Blues'
    
    total_df = pd.DataFrame()
    
    for_histogram = []
    
    for hspc_num in range(6):
        hspc_list = get_interaction_list(hspc_dirs[hspc_num], hspc_n[hspc_num], non_barcode_type, sorr)
        all_interactions = hspc_list[0].mean(axis = 1) + hspc_list[1].mean(axis = 1) + hspc_list[2].mean(axis = 1)
        for_histogram.extend(all_interactions)
        total_df[hspc_n[hspc_num]] = all_interactions

    if frequency: 
        for ind in total_df.index:
            if ind not in bm_freq_dict.keys():
                total_df = total_df.drop(ind)

        total_df = total_df.T
        for ind in total_df.columns:
            total_df[ind] *= (bm_freq_dict[ind]['Frequency']/100)
        
        total_df = total_df.T

        if imaging:
            for ind in total_df.index:
                if ind not in bm_distance.keys():
                    total_df = total_df.drop(ind)
                    
            total_df = total_df.T
            
            for ind in total_df.columns:
                for ct in ['hsc','mpp',"mep","gmp"]:
                    
                    total_df[ind][ct] *= (1/bm_distance[ind][ct.upper()])
            
            total_df = total_df.T
            total_df = total_df[['hsc','mpp','mep','gmp']]
    
    for_histogram = []
    for ex in total_df.values.tolist():
        for x in ex:
            for_histogram.append(x)
    
    xticks = [i.upper() for i in total_df.columns]
    
    if saving:
        if frequency:
            if imaging:
                total_df.to_csv("Figure_1_Pretty/supplemental_1_data_2_" + non_barcode_type + "_" + sorr + ".csv")
            else:
                total_df.to_csv("Figure_1_Pretty/supplemental_1_data_1_" + non_barcode_type + "_" + sorr + ".csv")
        else:
            total_df.to_csv("Figure_1_Pretty/supplemental_1_data_0_" + non_barcode_type + "_" + sorr + ".csv")
    
    if mask_lim:
        cg = sns.heatmap(total_df, mask = total_df < mask_lim, xticklabels = xticks, cmap = chosen_cmap,vmin = 0,  square = True, linewidth = 0.7)
    else:
        cg = sns.heatmap(total_df, xticklabels = xticks, cmap = chosen_cmap, square = True,vmax = vmax,vmin = 0, linewidth = 0.7)
    
    cg.set_xticklabels(cg.xaxis.get_majorticklabels(),rotation = 90)
    cg.set_yticklabels(cg.yaxis.get_majorticklabels(),rotation = 0)
    cg.add_patch(Rectangle((0,0), len(total_df.columns), len(total_df.index), fill = False, edgecolor = 'black', lw = 2))
    plt.show()

# Draw heatmaps of interaction intensity with corrections for frequency and spatial organization

sorr : 's' send OR 'r' receive <br>
non_barcode_type : ['Non-hematopoietic', 'Blood+Immune', 'HPC']

In [ ]:
NH_r = plot_population_hspcs_total('r', 'Non-hematopoietic',frequency = True, imaging = True)